### Aqui estava coletando apenas os 25 primeiros filmes, problema corrigido no notebook "ws_IMDb_2"

In [ ]:
import time, random
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Definindo "headers" para simular um navegador
# Alguns sites bloqueiam requisições que não parecem vir de um navegador real.
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0 Safari/537.36"
}

# Criando uma sessão persistente
# Reaproveita cookies/conexões e aplica headers em todas as requisições
session = requests.Session()
session.headers.update(headers)

# Função auxiliar para extrair o texto de um elemento do BeautifulSoup
# Retorna None se o elemento não existir (evita erros de 'NoneType')
def get_text(el):
    return el.get_text(strip=True) if el else None

# Função para fazer a requisição HTTP com várias tentativas (retries)
def fetch(url, retries=3, backoff=2):
    for i in range(retries):
        r = session.get(url, timeout=20) # timeout evita travar se o servidor não responder
        if r.status_code == 200: # se resposta OK, retorna imediatamente
            return r
        time.sleep(backoff ** i) # espera antes da próxima tentativa
    raise RuntimeError(f"Falha ao baixar: {url}") # se todas as tentativas falharem, lança erro

url = "https://www.imdb.com/chart/top/" # Definindo a URL do nosso alvo
response = fetch(url) # Verificando se a requisição foi bem-sucedida

print(f"Status Code: {response.status_code}") # Deve ser 200 se OK

Status Code: 200


In [ ]:
# Analisando (parsing) o conteúdo HTML com o BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Encontrando a lista de filmes
# A investigação mostrou que cada filme está em uma tag <li> dentro de uma <ul> com a classe "ipc-metadata-list"
filmes_data = []
lista_filmes = soup.find("ul", class_="ipc-metadata-list")

for filme_li in lista_filmes.find_all("li"):
    try:
        # Dados básicos do filme na primeira página
        # Extraindo Título, Ano e Nota
        titulo_tag = filme_li.find("h3", class_="ipc-title__text")
        if not titulo_tag:
            continue
        titulo = titulo_tag.text.split(". ", 1)[1]

        metadata = filme_li.find_all("span", class_="cli-title-metadata-item")
        ano = metadata[0].text if metadata else None

        nota_tag = filme_li.find("span", class_="ipc-rating-star")
        nota = get_text(nota_tag).split("(")[0].strip() if nota_tag else None

        # Extraindo o LINK para a página do filme específico
        link_relativo = titulo_tag.find_parent("a")["href"]
        link_completo = "https://www.imdb.com" + link_relativo

        # Acessando a página individual do filme para mais detalhes
        # Pausa de 1.5 a 2.5 segundos para não sobrecar
        time.sleep(random.uniform(1.5, 2.5))  # respeitar rate limit
        # Fazendo a requisição para a página individual do filme
        response_filme = fetch(link_completo)
        soup_filme = BeautifulSoup(response_filme.content, "html.parser")

        # Sinopse (vários fallbacks porque o data-testid muda às vezes)
        sinopse = None
        for sel in [
            '[data-testid="plot-xl"]',     # layout novo
            '[data-testid="plot-l"]',      # seu seletor original
            'span[data-testid="plot"]',    # fallback genérico
            'div[data-testid="storyline-plot-summary"] > div > div'  # storyline box
        ]:
            el = soup_filme.select_one(sel)
            if el:
                sinopse = get_text(el)
                break

        # Elenco principal
        elenco = []
        atores = soup_filme.select('a[data-testid="title-cast-item__actor"]')
        if not atores:
            # fallback: pega itens da lista de cast
            atores = soup_filme.select('[data-testid="title-cast-item"] a[href*="/name/"]')
        elenco = [get_text(a) for a in atores][:3]  # pegue os 3 principais

        # Diretor (com fallback se mudar a posição)
        diretor = None
        creditos_tags = soup_filme.find_all('ul', {'role': 'presentation', 'class': 'ipc-metadata-list'})
        if creditos_tags:
            a0 = creditos_tags[0].find('a')
            diretor = get_text(a0)

        filmes_data.append({
            "Título": titulo,
            "Ano": ano,
            "Nota": nota,
            "Sinopse": sinopse,
            "Diretor": diretor,
            "Elenco Principal": elenco
        })

    except Exception as e:
        print(f"Falha ao extrair dados de '{titulo if 'titulo' in locals() else 'Desconhecido'}': {e}")

print(f"Total de filmes extraídos: {len(filmes_data)}")
print("\nColeta finalizada!")
for filme in filmes_data[:5]:
    print(filme)


Coleta finalizada!
{'Título': 'Um Sonho de Liberdade', 'Ano': '1994', 'Nota': '9.3', 'Sinopse': 'A banker convicted of uxoricide forms a friendship over a quarter century with a hardened convict, while maintaining his innocence and trying to remain hopeful through simple compassion.', 'Diretor': 'Frank Darabont', 'Elenco Principal': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']}
{'Título': 'O Poderoso Chefão', 'Ano': '1972', 'Nota': '9.2', 'Sinopse': 'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.', 'Diretor': 'Francis Ford Coppola', 'Elenco Principal': ['Marlon Brando', 'Al Pacino', 'James Caan']}
{'Título': 'Batman: O Cavaleiro das Trevas', 'Ano': '2008', 'Nota': '9.1', 'Sinopse': 'When a menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman, James Gordon and Harvey Dent must work together to put an end to the madness.', 'Diretor': 'Christopher Nolan', 'Elenco Principal': ['Christian Ba

In [ ]:
df_filmes = pd.DataFrame(filmes_data)

caminho_arquivo = '../data/raw/imdb_top_250.csv'
df_filmes.to_csv(caminho_arquivo, index=False,
                                  sep = ';',
                                  decimal = ',',
                                  encoding = 'latin1'
)

print(f"\nDados salvos com sucesso em: {caminho_arquivo}")

df_filmes.head(10)


Dados salvos com sucesso em: ../data/raw/imdb_top_250.csv


,Título,Ano,Nota,Sinopse,Diretor,Elenco Principal
0,Um Sonho de Liberdade,1994,9.3,A banker convicted of uxoricide forms a friend...,Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton]"
1,O Poderoso Chefão,1972,9.2,The aging patriarch of an organized crime dyna...,Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan]"
2,Batman: O Cavaleiro das Trevas,2008,9.1,When a menace known as the Joker wreaks havoc ...,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart]"
3,O Poderoso Chefão: Parte II,1974,9.0,The early life and career of Vito Corleone in ...,Francis Ford Coppola,"[Al Pacino, Robert De Niro, Robert Duvall]"
4,12 Homens e uma Sentença,1957,9.0,The jury in a New York City murder trial is fr...,Sidney Lumet,"[Henry Fonda, Lee J. Cobb, Martin Balsam]"
5,O Senhor dos Anéis: O Retorno do Rei,2003,9.0,Gandalf and Aragorn lead the World of Men agai...,Peter Jackson,"[Elijah Wood, Viggo Mortensen, Ian McKellen]"
6,A Lista de Schindler,1993,9.0,"In German-occupied Poland during World War II,...",Steven Spielberg,"[Liam Neeson, Ralph Fiennes, Ben Kingsley]"
7,O Senhor dos Anéis: A Sociedade do Anel,2001,8.9,A meek Hobbit from the Shire and eight compani...,Peter Jackson,"[Elijah Wood, Ian McKellen, Orlando Bloom]"
8,Pulp Fiction: Tempo de Violência,1994,8.8,"The lives of two mob hitmen, a boxer, a gangst...",Quentin Tarantino,"[John Travolta, Uma Thurman, Samuel L. Jackson]"
9,Três Homens em Conflito,1966,8.8,A bounty-hunting scam joins two men in an unea...,Sergio Leone,"[Clint Eastwood, Eli Wallach, Lee Van Cleef]"


In [27]:
df_filmes.shape

(25, 6)